In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/leenalhossan/day-25/day25_project.csv


In [10]:
import numpy as np
import pandas as pd 

In [11]:
path="/kaggle/input/datasets/leenalhossan/day-25/day25_project.csv"
df= pd.read_csv(path)
print(df.columns)

Index(['price', 'sqft', 'rooms'], dtype='object')


In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold

path="/kaggle/input/datasets/leenalhossan/day-25/day25_project.csv"
df = pd.read_csv(path)

print("Initial Data:")
print(df.head())
print("\nData Summary:")
print(df.describe())

df["price_per_sqft"] = df["price"] / df["sqft"]
print("\nDomain Feature Added:")
print(df[["price", "sqft", "price_per_sqft"]].head())


print("\nVariance Comparison:")
print(df[["price", "sqft", "rooms", "price_per_sqft"]].var())

corr_matrix = df[["price", "sqft", "rooms", "price_per_sqft"]].corr()

print("\nCorrelation Matrix:")
print(corr_matrix)

print("\nCorrelation with price:")
print(corr_matrix["price"].sort_values(ascending=False))

selector = VarianceThreshold(threshold=0.0)
X = df[["price", "sqft", "rooms", "price_per_sqft"]]
X_selected = selector.fit_transform(X)

print("\nVariance Threshold Kept Features:")
print(selector.get_support())

def drop_correlated_features(df, threshold=0.9):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    return df.drop(columns=to_drop), to_drop

X_reduced, dropped_cols = drop_correlated_features(
    df[["price", "sqft", "rooms", "price_per_sqft"]],
    threshold=0.9
)

print("\nDropped Columns Due to High Correlation:")
print(dropped_cols)

print("Remaining Columns:")
print(X_reduced.columns.tolist())


target = "price"
features = ["sqft", "rooms", "price_per_sqft"]

print("\nLeakage Check:")

if target in features:
    print(" Target included in features!")

if "price_per_sqft" in features:
    print(" price_per_sqft derived from price → Potential Leakage!")

Initial Data:
    price  sqft  rooms
0  300000  1500      5
1  450000  2000      6
2  275000  1200      4
3  600000  2400      7

Data Summary:
               price         sqft     rooms
count       4.000000     4.000000  4.000000
mean   406250.000000  1775.000000  5.500000
std    150519.932235   531.507291  1.290994
min    275000.000000  1200.000000  4.000000
25%    293750.000000  1425.000000  4.750000
50%    375000.000000  1750.000000  5.500000
75%    487500.000000  2100.000000  6.250000
max    600000.000000  2400.000000  7.000000

Domain Feature Added:
    price  sqft  price_per_sqft
0  300000  1500      200.000000
1  450000  2000      225.000000
2  275000  1200      229.166667
3  600000  2400      250.000000

Variance Comparison:
price             2.265625e+10
sqft              2.825000e+05
rooms             1.666667e+00
price_per_sqft    4.210069e+02
dtype: float64

Correlation Matrix:
                   price      sqft     rooms  price_per_sqft
price           1.000000  0.981741